# Caching plots with context item attachments
This notebook demonstrates how to wrap any Plotly chart-building routine in a single cache-and-retrieve utility with TrendMiner context items. After you define your own make_fig() function (which loads data and returns a Plotly Figure), you call cache_context_plot(...) once—behind the scenes it will:
	1.	Check whether a previous JSON attachment exists and how old it is.
	2.	Regenerate and upload a fresh figure only if the cache is missing or stale.
	3.	Download and display the cached figure instantly when it’s up to date.

With this pattern, you get the speed of cached charts and the flexibility to redraw whenever your data changes—all in one clean, reusable function.

In [1]:
import os
import logging
from datetime import datetime, timedelta
from dateutil import parser
import pytz
import requests
import plotly.io as pio

def cache_context_plot(
    make_fig, 
    context_item_id, 
    server_url=None, 
    token=None, 
    filename="cached_plotly_output", 
    extension="json", 
    refresh_delta=timedelta(hours=1)
):
    """
    Cache a Plotly figure as a TrendMiner context‐item attachment,
    regenerating only if older than refresh_delta.
    """
    server = server_url or os.environ["KERNEL_SERVER_URL"]
    tok    = token     or os.environ["KERNEL_USER_TOKEN"]
    hdrs   = {"Authorization": f"Bearer {tok}"}

    # 1) list existing attachments
    resp = requests.get(
        f"{server}/context/data/{context_item_id}/attachments",
        headers=hdrs,
        params={"size": 1000}
    )
    resp.raise_for_status()
    att_id, mtime = None, None
    for itm in resp.json()["content"]:
        if itm["name"] == filename and itm["extension"] == extension:
            att_id = itm["identifier"]
            mtime  = parser.isoparse(itm["lastModifiedDate"])
            break

    # 2) decide whether to refresh
    now = pytz.utc.localize(datetime.utcnow())
    needs_refresh = (mtime is None) or ((mtime + refresh_delta) < now)

    if needs_refresh:
        logging.info("Regenerating and uploading fresh plot…")
        fig_json = pio.to_json(make_fig())

        # delete old if present
        if att_id:
            requests.delete(
                f"{server}/context/data/{context_item_id}/attachments/{att_id}",
                headers=hdrs
            ).raise_for_status()
            logging.info(f"Deleted old attachment {att_id}")

        # upload new
        upload_resp = requests.post(
            f"{server}/context/data/{context_item_id}/attachments",
            headers={**hdrs, "content-type": "application/octet-stream"},
            files={"file": (f"{filename}.{extension}", fig_json)},
            params={"name": filename, "extension": extension}
        )
        upload_resp.raise_for_status()
        logging.info("Uploaded new cache")
        return pio.from_json(fig_json)

    else:
        logging.info("Loading figure from cache…")
        dl_resp = requests.get(
            f"{server}/context/data/{context_item_id}/attachments/{att_id}/download",
            headers=hdrs
        )
        dl_resp.raise_for_status()
        raw = dl_resp.content.decode("utf-8", errors="ignore")

        # robustly extract JSON between first { and last }
        start = raw.find("{")
        end   = raw.rfind("}") + 1
        json_str = raw[start:end]

        return pio.from_json(json_str)

ModuleNotFoundError: No module named 'pytz'

In [ ]:
from trendminer.views.views import Views
import plotly.express as px

def make_fig():
    # load your data however you like…
    views = Views(TrendMinerClient(os.environ["KERNEL_USER_TOKEN"], os.environ["KERNEL_SERVER_URL"]))
    df = views.load_view("4b950668-3885-44e6-bf5d-598756836741")[0]
    df_long = (
        df.reset_index()
          .melt(id_vars="index",
                value_vars=["[CS]BA:CONC.1","[CS]BA:LEVEL.1","[CS]BA:TEMP.1"],
                var_name="Parameter", value_name="Value")
    )
    fig = px.line(df_long, x="index", y="Value", color="Parameter",
                  title="Concentration, Level & Temperature over Time")
    fig.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
    return fig

fig = cache_context_plot(
    make_fig,
    context_item_id="f078aa08-65c5-4247-96f9-ddef972cb73c",
    refresh_delta=timedelta(minutes=1)
)
fig.show()